# <span style='color:red'>Project 4.  Due December 1</span>

In this project we develop an updated first-order algorithm to construct a portfolio using intraday data; now with position bounds and using projected gradients.

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

##### We will have data involving $n$ assets, and use the first $T$ days of the data to compute the portfolio.
##### The computation will produce a weight $x_i$ for each asset $i = 1,...,n$, which could be long or short.
##### We assume that on each day, a position is taken at the open, and closed at noon.  So we define:
$$ p^o_{j,t} = \ \text{price of asset $j$ on day $t$ at the open}$$
$$ p^1_{j,t} = \ \text{price of asset $j$ on day $t$ at noon}$$
$$ r_{j,t} =  \ \frac{p^1_{j,t} - p^o_{j,t}}{p^o_{j,t}} = \ \text{return earned by asset $j$ on day $t$.}$$
$$ \bar r_j = \ \frac{1}{T} \sum_{t = 1}^T r_{j,t} = \ \text{average return earned by asset $j$.}$$

The assets that we will use are: ABR, AMZN, GS, NFLX, NIO, NVDA, TSLA, UBS.

#### The optimization problem to solve depends on two parameters: $\theta \ge 0$ and $\pi > 0$.

$$ \text{minimize} \ \left(-\sum_{j = 1}^n \bar r_j x_j\right) \ + \ \theta \left( \frac{1}{T} \sum_{t = 1}^{T}\left[\sum_{j = 1}^n (r_{j,t} -  \bar r_j)x_j\right]^\pi\right)$$

$$ \text{subject to} \ |x_j| \le 1 \ \text{for all names $j$}$$
#### Unlike what we did on project $2$, the risk term is **not** raised to the power $1/\pi$.


### <span style='color:red'> Develop a first-order method to address this computational problem.</span>

#### Your method should work with values of $T$ at least $100$.

#### Make sure your code works with $\pi = 2$ and $\pi = 4$.  For $\pi = 2$ use $\theta = 5$. You should be able to get meaningful problem behavior (i.e., nonzero portfolio).  And when $\pi = 4$ use $\theta = 1000$.
<br>

#### Suggestion: first apply the problem to one asset at a time, i.e., $n = 1$.  Then use $n = 2$.  And only then try the entire set of names.  And use $\pi = 2$ before trying $\pi = 4$.

### <span style='color:red'>**Extra credit**</span>
#### Suppose $x^*$ is the solution you compute above.  Next, solve the same optimization problem as above, except that you now have the constraint:
$$ \sum_j | x_j | \ \le \ 0.5*\sum_j |x^*_j |$$

Use a first-order barrier method, as covered in class, for this purpose.

### Please be sure to have completed all other parts of this project before trying this part.  "Extra credit" = only valid if other parts have been done.